In [ ]:
# --- Phase 1: fetch, align, and summarize PocketBase data ---

from pb_client import fetch_records
import pandas as pd

# 1. Pull your PocketBase data
glucose = pd.DataFrame(fetch_records("glucose"))
steps   = pd.DataFrame(fetch_records("steps"))

# 2. Convert timestamps to datetime
glucose["timestamp"] = pd.to_datetime(glucose["timestamp"])
steps["timestamp"]   = pd.to_datetime(steps["timestamp"])

# 3. Resample to 5-minute bins
#    (average glucose, sum of steps)
g = (
    glucose.set_index("timestamp")[["value_mgdl"]]   # <-- your actual field name
    .resample("5min")
    .mean()
    .rename(columns={"value_mgdl": "glucose"})
)
s = (
    steps.set_index("timestamp")[["steps"]]
    .resample("5min")
    .sum()
)

# 4. Merge and aggregate per day
daily = (
    g.join(s, how="outer")
    .resample("1D")
    .agg({
        "glucose": ["mean", "std"],
        "steps": ["sum"]
    })
)

# 5. Flatten column names and tidy up
daily.columns = ["glucose_mean", "glucose_std", "steps_sum"]
daily = daily.reset_index().rename(columns={"timestamp": "day"})

print("✅ Daily summary ready — showing last 10 days:\n")
print(daily.tail(10))


📡 Fetching glucose page 1...
📡 Fetching glucose page 2...
📡 Fetching glucose page 3...
📡 Fetching glucose page 4...
📡 Fetching glucose page 5...
📡 Fetching glucose page 6...
📡 Fetching glucose page 7...
📡 Fetching glucose page 8...
📡 Fetching glucose page 9...
📡 Fetching glucose page 10...
📡 Fetching glucose page 11...
📡 Fetching glucose page 12...
📡 Fetching glucose page 13...
📡 Fetching glucose page 14...
📡 Fetching glucose page 15...
📡 Fetching glucose page 16...
📡 Fetching glucose page 17...
📡 Fetching glucose page 18...
📡 Fetching glucose page 19...
📡 Fetching glucose page 20...
📡 Fetching glucose page 21...
📡 Fetching glucose page 22...
📡 Fetching glucose page 23...
📡 Fetching glucose page 24...
📡 Fetching glucose page 25...
📡 Fetching glucose page 26...
📡 Fetching glucose page 27...
📡 Fetching glucose page 28...
📡 Fetching glucose page 29...
📡 Fetching glucose page 30...
📡 Fetching glucose page 31...
📡 Fetching glucose page 32...
📡 Fetching glucose page 33...
📡 Fetching glucose 

KeyError: "Column(s) ['glucose'] do not exist"

In [4]:
print("g columns:", g.columns.tolist())
print("s columns:", s.columns.tolist())


g columns: ['value_mgdl']
s columns: ['steps']
